In [1]:
using PastaQ
using ITensors

In [2]:
# number of qubits
N = 5

# manually create a circuit to prepare GHZ state,
# or use built-in call `circuit = ghz(n)` 
circuit = Tuple[("H", 1)]
for j in 1:N-1
  push!(circuit, ("CX", (j, j+1)))
end

# run the circuit to obtain the output MPS
hilbert = qubits(N)
ψ = runcircuit(hilbert, circuit)


# sample projective measurements in the computational basis
@show getsamples(ψ, 5)

getsamples(ψ, 5) = [0 0 0 0 0; 1 1 1 1 1; 1 1 1 1 1; 1 1 1 1 1; 1 1 1 1 1]


5×5 Matrix{Int64}:
 0  0  0  0  0
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1
 1  1  1  1  1

In [ ]:
ψ = runcircuit(hilbert, circuit)
@which getsamples(ψ, 5)

In [10]:
ψ = runcircuit(hilbert, circuit)
ρ = outer(ψ, prime(ψ))
@which sample(ρ)

sample(M::MPO) in ITensors at /Users/alcalde/workprojects/forks/ITensors.jl/src/mps/mpo.jl:863

In [17]:
noisemodel = (1 => ("depolarizing", (p = 0.1,)), 
              2 => ("depolarizing", (p = 0.5,)))

circuit2 = Tuple[("H", 1)]
for j in 1:N-1
  push!(circuit2, ("CX", (j, j+1)))
end

ψ = randomMPS(hilbert, linkdims=10)

runcircuit(ψ, circuit2; noismodel=noisemodel)

ψ = randomMPS(hilbert, linkdims=10)
runcircuit(ψ, circuit2; noise = noisemodel)

MPO
[1] ((dim=2|id=43|"Qubit,Site,n=1")', (dim=2|id=43|"Qubit,Site,n=1"), (dim=4|id=685|"Link,n=1"))
[2] ((dim=2|id=171|"Qubit,Site,n=2")', (dim=4|id=685|"Link,n=1"), (dim=2|id=171|"Qubit,Site,n=2"), (dim=16|id=129|"Link,n=1"))
[3] ((dim=2|id=825|"Qubit,Site,n=3")', (dim=16|id=129|"Link,n=1"), (dim=2|id=825|"Qubit,Site,n=3"), (dim=16|id=842|"Link,n=1"))
[4] ((dim=2|id=976|"Qubit,Site,n=4")', (dim=16|id=842|"Link,n=1"), (dim=2|id=976|"Qubit,Site,n=4"), (dim=4|id=989|"Link,n=1"))
[5] ((dim=4|id=989|"Link,n=1"), (dim=2|id=165|"Qubit,Site,n=5")', (dim=2|id=165|"Qubit,Site,n=5"))


In [4]:
# define a noise model with different error rates for
# one- and two-qubit gates
noisemodel = (1 => ("depolarizing", (p = 0.1,)), 
              2 => ("depolarizing", (p = 0.5,)))

# run a noisy circuit
ρ = runcircuit(hilbert, circuit; noise = noisemodel)
@show fidelity(ψ, ρ)
@show getsamples(ρ, 5)

fidelity(ψ, ρ) = 0.11689744855967078
getsamples(ρ, 5) = [0 0 1 1 1; 0 0 0 1 0; 0 1 1 1 1; 0 1 1 0 0; 0 1 1 1 1]


5×5 Matrix{Int64}:
 0  0  1  1  1
 0  0  0  1  0
 0  1  1  1  1
 0  1  1  0  0
 0  1  1  1  1

MPO
[1] ((dim=2|id=43|"Qubit,Site,n=1")', (dim=2|id=43|"Qubit,Site,n=1"), (dim=4|id=324|"Link,n=1"))
[2] ((dim=2|id=171|"Qubit,Site,n=2")', (dim=4|id=324|"Link,n=1"), (dim=2|id=171|"Qubit,Site,n=2"), (dim=16|id=191|"Link,n=1"))
[3] ((dim=2|id=825|"Qubit,Site,n=3")', (dim=16|id=191|"Link,n=1"), (dim=2|id=825|"Qubit,Site,n=3"), (dim=16|id=31|"Link,n=1"))
[4] ((dim=2|id=976|"Qubit,Site,n=4")', (dim=16|id=31|"Link,n=1"), (dim=2|id=976|"Qubit,Site,n=4"), (dim=4|id=348|"Link,n=1"))
[5] ((dim=4|id=348|"Link,n=1"), (dim=2|id=165|"Qubit,Site,n=5")', (dim=2|id=165|"Qubit,Site,n=5"))


MPO
[1] ((dim=2|id=43|"Qubit,Site,n=1")', (dim=2|id=43|"Qubit,Site,n=1"), (dim=4|id=123|"Link,n=1"))
[2] ((dim=2|id=171|"Qubit,Site,n=2")', (dim=4|id=123|"Link,n=1"), (dim=2|id=171|"Qubit,Site,n=2"), (dim=16|id=555|"Link,n=1"))
[3] ((dim=2|id=825|"Qubit,Site,n=3")', (dim=16|id=555|"Link,n=1"), (dim=2|id=825|"Qubit,Site,n=3"), (dim=16|id=370|"Link,n=1"))
[4] ((dim=2|id=976|"Qubit,Site,n=4")', (dim=16|id=370|"Link,n=1"), (dim=2|id=976|"Qubit,Site,n=4"), (dim=4|id=759|"Link,n=1"))
[5] ((dim=4|id=759|"Link,n=1"), (dim=2|id=165|"Qubit,Site,n=5")', (dim=2|id=165|"Qubit,Site,n=5"))


In [3]:
# quantum processes can also be obtained.
# unitary MPO
U = runcircuit(circuit; process = true)
# Choi matrix
Λ = runcircuit(circuit; process = true, noise = noisemodel)

MPO
[1] ((dim=2|id=605|"Output,Qubit,Site,n=1")', (dim=2|id=605|"Input,Qubit,Site,n=1")', (dim=2|id=605|"Input,Qubit,Site,n=1"), (dim=2|id=605|"Output,Qubit,Site,n=1"), (dim=5|id=624|"Link,n=1"))
[2] ((dim=2|id=982|"Output,Qubit,Site,n=2")', (dim=5|id=624|"Link,n=1"), (dim=2|id=982|"Input,Qubit,Site,n=2")', (dim=2|id=982|"Input,Qubit,Site,n=2"), (dim=2|id=982|"Output,Qubit,Site,n=2"), (dim=5|id=343|"Link,n=1"))
[3] ((dim=2|id=490|"Output,Qubit,Site,n=3")', (dim=5|id=343|"Link,n=1"), (dim=2|id=490|"Input,Qubit,Site,n=3")', (dim=2|id=490|"Input,Qubit,Site,n=3"), (dim=2|id=490|"Output,Qubit,Site,n=3"), (dim=5|id=620|"Link,n=1"))
[4] ((dim=2|id=288|"Output,Qubit,Site,n=4")', (dim=5|id=620|"Link,n=1"), (dim=2|id=288|"Input,Qubit,Site,n=4")', (dim=2|id=288|"Input,Qubit,Site,n=4"), (dim=2|id=288|"Output,Qubit,Site,n=4"), (dim=5|id=369|"Link,n=1"))
[5] ((dim=5|id=369|"Link,n=1"), (dim=2|id=837|"Output,Qubit,Site,n=5")', (dim=2|id=837|"Input,Qubit,Site,n=5")', (dim=2|id=837|"Input,Qubit,Site,n=